### Import necessary packages:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.wcs import WCS
from reproject import reproject_interp
import numpy as np
import astropy.io.fits as fits
import healpy as hp
from matplotlib import cm
from astropy_healpix import HEALPix
from astropy.coordinates import SkyCoord, ICRS, Galactic
import astropy.units as u

### Read in Dwingeloo data:

In [ ]:
hduQ = fits.open('/srv/data/dwingeloo/DWING_820_EQU_Q.HPX.fits')
hdrQ = hduQ[0].header
dataQ = hduQ[0].data[0]
print(dataQ.shape)

hduU = fits.open('/srv/data/dwingeloo/DWING_820_EQU_U.HPX.fits')
hdrU = hduU[0].header
dataU = hduU[0].data[0]
print(dataU.shape)

### Pick the range of RA and dec to plot, and the gridding pixel size

In [ ]:
# Sample a grid in RA/Dec
RA_min = 0
RA_max = 360
dec_min = -10
dec_max = 90
delta = 0.5

### Set up the HEALpix projection, grid and coordinates
#### NOTE: if all files have the same NSIDE, just need to do this once

In [ ]:
# Healpix parameters:
nside = hdrQ['NSIDE']
order = hdrQ['ORDERING']
hpx = HEALPix(nside=nside, order=order, frame=ICRS())

# Number of pixels along each axis:
numRA = int((RA_max-RA_min)/delta)
numdec = int((dec_max-dec_min)/delta)

# 1D and 2D RA and dec grids:
ra = np.linspace(RA_min, RA_max, numRA) * u.deg
dec = np.linspace(dec_min, dec_max, numdec) * u.deg
ra_grid, dec_grid = np.meshgrid(ra, dec)

# Set up Astropy coordinate objects
coords = SkyCoord(ra_grid.ravel(), dec_grid.ravel(), frame='icrs')

### Reproject the HEALpix values onto the 2D grid for each map:

In [ ]:
Qmap = hpx.interpolate_bilinear_skycoord(coords, dataQ)
Qmap = Qmap.reshape((numdec, numRA))[:,::-1]
print(Qmap.shape)

Umap = hpx.interpolate_bilinear_skycoord(coords, dataU)
Umap = Umap.reshape((numdec, numRA))[:,::-1]
print(Umap.shape)


In [ ]:
hdr_new = fits.Header.fromstring("""
NAXIS   =                    2
CUNIT1  = 'deg     '
CUNIT2  = 'deg     '
""", sep='\n')

hdr_new['COORDSYS']  = 'icrs'
hdr_new['NAXIS1']  = Qmap.shape[1] 
hdr_new['NAXIS2']  = Qmap.shape[0]

hdr_new['CTYPE1']  = 'RA---CAR'
hdr_new['CRPIX1']  = Qmap.shape[1]/2.+0.5 
hdr_new['CRVAL1']  = (RA_min+RA_max)/2.          
hdr_new['CDELT1']  = -delta

hdr_new['CTYPE2']  = 'DEC--CAR'
hdr_new['CRPIX2']  = Qmap.shape[0]/2.+0.5
hdr_new['CRVAL2']  = (dec_min+dec_max)/2.
hdr_new['CDELT2']  = delta

print(repr(hdr_new))

In [ ]:
fits.writeto('/home/aordog/DATA/fits_map_test.fits',Qmap,hdr_new)

### Plots:

In [ ]:
plt.figure(figsize=(20, 4))
im = plt.imshow(Qmap, extent=[RA_max, RA_min, dec_min, dec_max], cmap=plt.cm.RdBu_r, 
                origin='lower', aspect='equal',vmin=-1,vmax=1)
plt.colorbar(im)
plt.xlabel('Right ascension (deg)')
plt.ylabel('Declination (deg)')
plt.title('Stokes Q')
plt.show()

plt.figure(figsize=(20, 4))
im = plt.imshow(Umap, extent=[RA_max, RA_min, dec_min, dec_max], cmap=plt.cm.RdBu_r, 
                origin='lower', aspect='equal',vmin=-1,vmax=1)
plt.colorbar(im)
plt.xlabel('Right ascension (deg)')
plt.ylabel('Declination (deg)')
plt.title('Stokes U')
plt.show()

In [ ]:
l_list = []
b_list = []
ra_list = []
dec_list = []
PI_list = []
gal_theta_list = []
eq_theta_list = []
with open('/srv/data/dwingeloo_data/original/out820') as f:
    for line in f:
        line = f.readline()
        l_list.append(float(line[0:7]))
        b_list.append(float(line[7:13]))
        ra_list.append(float(line[13:19]))
        dec_list.append(float(line[19:25]))
        PI_list.append(float(line[25:31]))
        gal_theta_list.append(float(line[31:37]))
        eq_theta_list.append(float(line[37:43]))
f.close()
ra_arr = np.asarray(ra_list)
dec_arr = np.asarray(dec_list)

dec_arr2 = dec_arr.copy()
dec_arr2[np.where(dec_arr>=270.)] = dec_arr2[np.where(dec_arr>=270.)]-360.

Qpts = np.asarray(PI_list)*np.cos((2*np.asarray(eq_theta_list))*np.pi/180)
Upts = np.asarray(PI_list)*np.sin((2*np.asarray(eq_theta_list))*np.pi/180)



In [ ]:
plt.figure(figsize=(20, 6))
im = plt.imshow(Qmap, extent=[RA_max, RA_min, dec_min, dec_max], cmap=plt.cm.RdBu_r, 
                origin='lower', aspect='equal',vmin=-1,vmax=1)
plt.scatter(ra_arr,dec_arr2,c=Qpts,vmin=-1,vmax=1,cmap=plt.cm.RdBu_r,s=30,edgecolors='k')
plt.colorbar(im)
plt.xlabel('Right ascension (deg)')
plt.ylabel('Declination (deg)')
plt.title('Stokes Q')
plt.show()

plt.figure(figsize=(20, 6))
im = plt.imshow(Umap, extent=[RA_max, RA_min, dec_min, dec_max], cmap=plt.cm.RdBu_r, 
                origin='lower', aspect='equal',vmin=-1,vmax=1)
plt.scatter(ra_arr,dec_arr2,c=Upts,vmin=-1,vmax=1,cmap=plt.cm.RdBu_r,s=30,edgecolors='k')
plt.colorbar(im)
plt.xlabel('Right ascension (deg)')
plt.ylabel('Declination (deg)')
plt.title('Stokes U')
plt.show()

In [ ]:
plt.figure(figsize=(20, 6))
im = plt.imshow(0.5*np.arctan2(Umap,Qmap)*180./np.pi, extent=[RA_max, RA_min, dec_min, dec_max], cmap=plt.cm.hsv, 
                origin='lower', aspect='equal',vmin=-90,vmax=90)
#plt.scatter(ra_arr,dec_arr2,c=0.5*np.arctan2(Upts,Qpts)*180./np.pi,vmin=-90,vmax=90,cmap=plt.cm.hsv,s=40,edgecolors='k')
plt.colorbar(im)
plt.xlabel('Right ascension (deg)')
plt.ylabel('Declination (deg)')
plt.title('PA')
plt.show()

In [ ]:
plt.scatter(eq_theta_list,0.5*np.arctan2(Upts,Qpts)*180./np.pi)
plt.grid()

In [ ]:
from astropy.coordinates import angular_separation

qwcs = WCS(hdr_new)

print(Qmap.shape[1])
print(Qmap.shape[0])
print(repr(qwcs))

c = SkyCoord.from_pixel(np.arange(Qmap.shape[1])[:,np.newaxis], np.arange(Qmap.shape[0])[np.newaxis,:], qwcs.celestial)
radius = 1 #* u.deg
drao_q_avg = []
drao_u_avg = []
for i in np.arange(len(ra_arr)):
    #print(ra_arr[i])
    #print(c.icrs.ra.deg)
    sep = angular_separation(np.radians(ra_arr[i]), np.radians(dec_arr2[i]), c.icrs.ra.radian, c.icrs.dec.radian)*180/np.pi
    #print(type(sep))
    idx = np.where(sep <= radius)
    #print(i)
    #print(ra_arr[i],dec_arr2[i])
    #print(c.icrs.ra.deg[idx])
    #print(c.icrs.dec.deg[idx])
    #print('')
    drao_q_avg.append(np.nanmean(Qmap[idx[1], idx[0]]))
    drao_u_avg.append(np.nanmean(Umap[idx[1], idx[0]]))  

In [ ]:
print(drao_q_avg)